In [20]:
import json
import numpy as np
import pandas as pd
import os
import sys
from pathlib import Path

ROOT = Path.cwd()
if not (ROOT / "pkg").exists():
    if (ROOT.parent / "pkg").exists():
        ROOT = ROOT.parent
    else:
        raise RuntimeError("Не найден корень репозитория (папка pkg)")

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from pkg.metadata import compute_metadata

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [21]:
ANNOTATIONS_DIR = ROOT / "experiments" / "data" / "labels"
OUTPUT_DIR = ROOT / "experiments" / "results" / "classification_dataset"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_CSV = OUTPUT_DIR / "teeth_classification_dataset.csv"

print(f"Директория с аннотациями: {ANNOTATIONS_DIR}")
print(f"Директория для результатов: {OUTPUT_DIR}")
print(f"Выходной файл: {OUTPUT_CSV}")

Директория с аннотациями: ../experiments/data/labels
Директория для результатов: ../experiments/results/classification_dataset
Выходной файл: ../experiments/results/classification_dataset\teeth_classification_dataset.csv


In [22]:
def polygon_to_bbox(points):
    """
    Конвертирует полигон в bounding box.
    :param points: Список точек [[x1, y1], [x2, y2], ...]
    :return: [x_min, y_min, x_max, y_max]
    """
    points = np.array(points)
    x_min = np.min(points[:, 0])
    y_min = np.min(points[:, 1])
    x_max = np.max(points[:, 0])
    y_max = np.max(points[:, 1])
    return [x_min, y_min, x_max, y_max]

In [23]:
json_files = list(Path(ANNOTATIONS_DIR).glob("*.json"))

print(f"Найдено файлов с аннотациями: {len(json_files)}")

Найдено файлов с аннотациями: 2066


In [24]:
all_records = []

for json_path in json_files:
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            annotation_data = json.load(f)
        
        img_width = annotation_data.get('imageWidth', 2440)
        img_height = annotation_data.get('imageHeight', 1280)
        
        bboxes = []
        labels = []
        
        for shape in annotation_data.get('shapes', []):
            tooth_label = shape['label']
            points = shape['points']
            
            bbox = polygon_to_bbox(points)
            
            bboxes.append(bbox)
            labels.append(tooth_label)
        
        if len(bboxes) == 0:
            print(f"Предупреждение: {json_path.name} - нет зубов")
            continue
        
        metadata_list = compute_metadata(bboxes, image_width=img_width)
        
        for i, meta in enumerate(metadata_list):
            record = {
                'tooth_label': labels[i],
                
                'quadrant': meta['quadrant'],
                'x_norm': meta['x_norm'],
                'y_norm': meta['y_norm'],
                'aspect_ratio': meta['aspect_ratio'],
                'neighbours_right': meta['neighbours_right'],
                'neighbours_top': meta['neighbours_top'],
            }
            all_records.append(record)
        
        print(f"Обработано: {json_path.name} - {len(bboxes)} зубов")
        
    except Exception as e:
        print(f"Ошибка при обработке {json_path.name}: {e}")
        continue

print(f"\nВсего собрано записей: {len(all_records)}")

Обработано: 1000.json - 32 зубов
Обработано: 1001.json - 28 зубов
Обработано: 1002.json - 28 зубов
Обработано: 1003.json - 26 зубов
Обработано: 1004.json - 26 зубов
Обработано: 1005.json - 26 зубов
Обработано: 1006.json - 21 зубов
Обработано: 1007.json - 26 зубов
Обработано: 1008.json - 24 зубов
Обработано: 1009.json - 24 зубов
Обработано: 1010.json - 22 зубов
Обработано: 1011.json - 22 зубов
Обработано: 1012.json - 22 зубов
Обработано: 1013.json - 22 зубов
Обработано: 1014.json - 29 зубов
Обработано: 1015.json - 29 зубов
Обработано: 1016.json - 8 зубов
Обработано: 1017.json - 8 зубов
Обработано: 1018.json - 8 зубов
Обработано: 1019.json - 8 зубов
Обработано: 1020.json - 28 зубов
Обработано: 1021.json - 28 зубов
Обработано: 1022.json - 25 зубов
Обработано: 1023.json - 25 зубов
Обработано: 1024.json - 23 зубов
Обработано: 1025.json - 23 зубов
Обработано: 1026.json - 23 зубов
Обработано: 1027.json - 23 зубов
Обработано: 1028.json - 23 зубов
Обработано: 1029.json - 23 зубов
Обработано: 10

Обработано: 2170.json - 32 зубов
Обработано: 2172.json - 22 зубов
Обработано: 2173.json - 22 зубов
Обработано: 2174.json - 23 зубов
Обработано: 2175.json - 30 зубов
Обработано: 2176.json - 30 зубов
Обработано: 2177.json - 30 зубов
Обработано: 2178.json - 29 зубов
Обработано: 2179.json - 27 зубов
Обработано: 2180.json - 27 зубов
Обработано: 2181.json - 27 зубов
Обработано: 2182.json - 27 зубов
Обработано: 2183.json - 30 зубов
Обработано: 2184.json - 25 зубов
Обработано: 2185.json - 25 зубов
Обработано: 2186.json - 25 зубов
Обработано: 2187.json - 25 зубов
Обработано: 2188.json - 27 зубов
Обработано: 2189.json - 27 зубов
Обработано: 2190.json - 27 зубов
Обработано: 2191.json - 24 зубов
Обработано: 2192.json - 24 зубов
Обработано: 2193.json - 24 зубов
Обработано: 2194.json - 22 зубов
Обработано: 2195.json - 30 зубов
Обработано: 2196.json - 30 зубов
Обработано: 2197.json - 31 зубов
Обработано: 2198.json - 25 зубов
Обработано: 2199.json - 26 зубов
Обработано: 2200.json - 26 зубов
Обработано

Обработано: 4948.json - 32 зубов
Обработано: 4949.json - 25 зубов
Обработано: 4950.json - 25 зубов
Обработано: 4951.json - 30 зубов
Обработано: 4952.json - 27 зубов
Обработано: 4953.json - 32 зубов
Обработано: 4954.json - 31 зубов
Обработано: 4955.json - 32 зубов
Обработано: 4956.json - 27 зубов
Обработано: 4957.json - 32 зубов
Обработано: 4958.json - 32 зубов
Обработано: 4959.json - 31 зубов
Обработано: 4960.json - 26 зубов
Обработано: 4961.json - 32 зубов
Обработано: 4962.json - 28 зубов
Обработано: 4963.json - 28 зубов
Обработано: 4964.json - 31 зубов
Обработано: 4966.json - 32 зубов
Обработано: 4967.json - 31 зубов
Обработано: 4969.json - 29 зубов
Обработано: 4971.json - 32 зубов
Обработано: 4973.json - 32 зубов
Обработано: 4974.json - 32 зубов
Обработано: 4975.json - 28 зубов
Обработано: 4976.json - 28 зубов
Обработано: 4977.json - 32 зубов
Обработано: 4978.json - 32 зубов
Обработано: 4979.json - 32 зубов
Обработано: 4981.json - 25 зубов
Обработано: 4982.json - 31 зубов
Обработано

Обработано: 5435.json - 28 зубов
Обработано: 5436.json - 32 зубов
Обработано: 5438.json - 30 зубов
Обработано: 5439.json - 28 зубов
Обработано: 5440.json - 20 зубов
Обработано: 5441.json - 29 зубов
Обработано: 5442.json - 31 зубов
Обработано: 5443.json - 29 зубов
Обработано: 5444.json - 31 зубов
Обработано: 5446.json - 31 зубов
Обработано: 5447.json - 32 зубов
Обработано: 5448.json - 32 зубов
Обработано: 5449.json - 30 зубов
Обработано: 5450.json - 29 зубов
Обработано: 5451.json - 31 зубов
Обработано: 5452.json - 32 зубов
Обработано: 5453.json - 32 зубов
Обработано: 5454.json - 32 зубов
Обработано: 5456.json - 30 зубов
Обработано: 5458.json - 22 зубов
Обработано: 5459.json - 31 зубов
Обработано: 5460.json - 28 зубов
Обработано: 5461.json - 26 зубов
Обработано: 5462.json - 29 зубов
Обработано: 5464.json - 28 зубов
Обработано: 5465.json - 28 зубов
Обработано: 5466.json - 30 зубов
Обработано: 5467.json - 30 зубов
Обработано: 5468.json - 31 зубов
Обработано: 5469.json - 31 зубов
Обработано

Обработано: 8040.json - 27 зубов
Обработано: 8041.json - 30 зубов
Обработано: 8042.json - 18 зубов
Обработано: 8043.json - 18 зубов
Обработано: 8044.json - 26 зубов
Обработано: 8045.json - 28 зубов
Обработано: 8046.json - 27 зубов
Обработано: 8047.json - 19 зубов
Обработано: 8048.json - 29 зубов
Обработано: 8049.json - 28 зубов
Обработано: 8050.json - 20 зубов
Обработано: 8051.json - 25 зубов
Обработано: 8052.json - 32 зубов
Обработано: 8053.json - 28 зубов
Обработано: 8054.json - 28 зубов
Обработано: 8055.json - 28 зубов
Обработано: 8056.json - 6 зубов
Обработано: 8057.json - 29 зубов
Обработано: 8058.json - 31 зубов
Обработано: 8059.json - 24 зубов
Обработано: 8060.json - 30 зубов
Обработано: 8061.json - 26 зубов
Обработано: 8062.json - 9 зубов
Обработано: 8063.json - 26 зубов
Обработано: 8064.json - 13 зубов
Обработано: 8065.json - 13 зубов
Обработано: 8066.json - 31 зубов
Обработано: 8067.json - 29 зубов
Обработано: 8068.json - 23 зубов
Обработано: 8069.json - 26 зубов
Обработано: 

Обработано: cw1157.json - 32 зубов
Обработано: cw1158.json - 32 зубов
Обработано: cw1159.json - 32 зубов
Обработано: cw1160.json - 32 зубов
Обработано: cw1161.json - 30 зубов
Обработано: cw1162.json - 31 зубов
Обработано: cw1163.json - 31 зубов
Обработано: cw1164.json - 27 зубов
Обработано: cw1165.json - 21 зубов
Обработано: cw1166.json - 29 зубов
Обработано: cw1167.json - 30 зубов
Обработано: cw1168.json - 31 зубов
Обработано: cw1169.json - 29 зубов
Обработано: cw1170.json - 30 зубов
Обработано: cw1171.json - 30 зубов
Обработано: cw1172.json - 32 зубов
Обработано: cw1173.json - 31 зубов
Обработано: cw1174.json - 32 зубов
Обработано: cw1175.json - 32 зубов
Обработано: cw1176.json - 24 зубов
Обработано: cw1177.json - 32 зубов
Обработано: cw1178.json - 30 зубов
Обработано: cw1179.json - 30 зубов
Обработано: cw1180.json - 32 зубов
Обработано: cw1181.json - 30 зубов
Обработано: cw1182.json - 21 зубов
Обработано: cw1183.json - 31 зубов
Обработано: cw1184.json - 27 зубов
Обработано: cw1185.j

Обработано: cw1459.json - 25 зубов
Обработано: cw1460.json - 32 зубов
Обработано: cw1461.json - 30 зубов
Обработано: cw1462.json - 32 зубов
Обработано: cw1463.json - 32 зубов
Обработано: cw1464.json - 32 зубов
Обработано: cw1465.json - 28 зубов
Обработано: cw1466.json - 32 зубов
Обработано: cw1467.json - 29 зубов
Обработано: cw1468.json - 32 зубов
Обработано: cw1469.json - 30 зубов
Обработано: cw1470.json - 22 зубов
Обработано: cw1471.json - 32 зубов
Обработано: cw1472.json - 32 зубов
Обработано: cw1473.json - 30 зубов
Обработано: cw1474.json - 31 зубов
Обработано: cw1475.json - 32 зубов
Обработано: cw1476.json - 32 зубов
Обработано: cw1477.json - 30 зубов
Обработано: cw1478.json - 32 зубов
Обработано: cw1479.json - 30 зубов
Обработано: cw1480.json - 32 зубов
Обработано: cw1481.json - 32 зубов
Обработано: cw1482.json - 24 зубов
Обработано: cw1483.json - 32 зубов
Обработано: cw1484.json - 31 зубов
Обработано: cw1485.json - 29 зубов
Обработано: cw1486.json - 29 зубов
Обработано: cw1487.j

Обработано: cw1699.json - 32 зубов
Обработано: cw1700.json - 31 зубов
Обработано: cw7300.json - 32 зубов
Обработано: cw7301.json - 31 зубов
Обработано: cw7302.json - 32 зубов
Обработано: cw7303.json - 29 зубов
Обработано: cw7304.json - 29 зубов
Обработано: cw7305.json - 32 зубов
Обработано: cw7306.json - 31 зубов
Обработано: cw7307.json - 28 зубов
Обработано: cw7308.json - 30 зубов
Обработано: cw7309.json - 32 зубов
Обработано: cw7310.json - 30 зубов
Обработано: cw7311.json - 31 зубов
Обработано: cw7312.json - 31 зубов
Обработано: cw7313.json - 32 зубов
Обработано: cw7314.json - 27 зубов
Обработано: cw7315.json - 32 зубов
Обработано: cw7316.json - 30 зубов
Обработано: cw7317.json - 31 зубов
Обработано: cw7318.json - 32 зубов
Обработано: cw7319.json - 30 зубов
Обработано: cw7320.json - 31 зубов
Обработано: cw7321.json - 27 зубов
Обработано: cw7322.json - 32 зубов
Обработано: cw7323.json - 28 зубов
Обработано: cw7324.json - 32 зубов
Обработано: cw7325.json - 30 зубов
Обработано: cw7326.j

In [26]:
df = pd.DataFrame(all_records)

print("\n" + "="*60)
print("СТАТИСТИКА ДАТАСЕТА")
print("="*60)

print(f"\nВсего зубов: {len(df)}")

print(f"\nРаспределение по номерам зубов:")
print(df['tooth_label'].value_counts().sort_index())

print(f"\nРаспределение по квадрантам:")
quadrant_names = {1: 'Upper-Right', 2: 'Upper-Left', 3: 'Lower-Left', 4: 'Lower-Right'}
for q in sorted(df['quadrant'].unique()):
    count = len(df[df['quadrant'] == q])
    print(f"  Квадрант {q} ({quadrant_names.get(q, 'Unknown')}): {count} зубов")

print(f"\nСтатистика признаков:")
print(df[['x_norm', 'y_norm', 'aspect_ratio', 'neighbours_right', 'neighbours_top']].describe())

print("\nПример данных (первые 10 записей):")
display(df.head(10))


СТАТИСТИКА ДАТАСЕТА

Всего зубов: 58269

Распределение по номерам зубов:
tooth_label
11    1978
12    1993
13    2017
14    1924
15    1845
16    1795
17    1812
18    1113
21    1975
22    1996
23    2024
24    1931
25    1838
26    1790
27    1837
28    1115
31    1957
32    1963
33    2038
34    1967
35    1876
36    1713
37    1759
38    1273
41    1946
42    1971
43    2041
44    1980
45    1898
46    1708
47    1809
48    1334
91      53
Name: count, dtype: int64

Распределение по квадрантам:
  Квадрант 1 (Upper-Right): 14689 зубов
  Квадрант 2 (Upper-Left): 14654 зубов
  Квадрант 3 (Lower-Left): 14397 зубов
  Квадрант 4 (Lower-Right): 14529 зубов

Статистика признаков:
             x_norm        y_norm  aspect_ratio  neighbours_right  neighbours_top
count  58269.000000  58269.000000  58269.000000      58269.000000    58269.000000
mean       0.498682      0.508160      2.463617         13.928676       13.907858
std        0.242613      0.221958      0.928761          8.569526   

,tooth_label,quadrant,x_norm,y_norm,aspect_ratio,neighbours_right,neighbours_top
0,12,1,0.404161,0.284247,2.926829,20,4
1,13,1,0.329029,0.274829,1.829932,22,2
2,14,1,0.287817,0.327055,2.018349,24,7
3,15,1,0.240582,0.347603,2.288889,25,9
4,16,1,0.186372,0.386130,1.561983,27,14
5,17,1,0.118848,0.379281,1.916667,29,13
6,18,1,0.055445,0.404110,1.833333,31,15
7,42,4,0.455835,0.702055,2.000000,17,20
8,43,4,0.405395,0.764555,2.334124,19,23
9,44,4,0.354073,0.773973,2.975610,21,26


In [27]:
df.to_csv(OUTPUT_CSV, index=False, encoding='utf-8')

print(f"\n" + "="*60)
print("ДАТАСЕТ УСПЕШНО СОЗДАН!")
print("="*60)
print(f"\nФайл сохранен: {OUTPUT_CSV}")
print(f"Всего записей: {len(df)}")
print(f"Количество признаков: {len(df.columns) - 2}")
print(f"Признаки: {list(df.columns[:-2])}")
print(f"Целевая переменная: tooth_label")
print("="*60)

summary = {
    "total_teeth": len(df),
    "total_images": df['source_file'].nunique(),
    "label_distribution": df['tooth_label'].value_counts().to_dict(),
    "quadrant_distribution": df['quadrant'].value_counts().to_dict(),
    "features": list(df.columns[:-2]),
    "target": "tooth_label",
    "output_file": OUTPUT_CSV
}

summary_path = os.path.join(OUTPUT_DIR, "dataset_summary.json")
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print(f"\nСводная статистика сохранена: {summary_path}")


ДАТАСЕТ УСПЕШНО СОЗДАН!

Файл сохранен: ../experiments/results/classification_dataset\teeth_classification_dataset.csv
Всего записей: 58269
Количество признаков: 5
Признаки: ['tooth_label', 'quadrant', 'x_norm', 'y_norm', 'aspect_ratio']
Целевая переменная: tooth_label


KeyError: 'source_file'